# Low Frequency Receiver (LFR)

<div>
<center>
<img src="resources/solar-orbiter-landscape-image-astrium.jpg" width="800" align="center">
</center>
</div>

<div>
<center>
<img src="resources/logo_LPP.png" width="200" align="left">
</center>
</div>

<p align="center">
  <b>Alexis Jeandet &lt;alexis.jeandet@lpp.polytechnique.fr&gt;:</b><br>
  <a href="https://github.com/jeandet">Github:jeandet</a> |
  <a href="https://gitter.im/jeandet">Gitter:jeandet</a> |
  <a href="https://freenode.net/">IRC@freenode:jeandet</a>
  <br>
</p>


# Laboratory of Plasma Physics (LPP)


# Laboratory of Plasma Physics (LPP)

- Since 2009, fusion between LPTP and CETP
- Around 110+ people 
- 3 Scientific teams, [Fusion, Cold, Space] Plasmas 

# Laboratory of Plasma Physics (LPP)


- Plasmas are used in many daily applications, TVs, lights, energy production, air cleaning, IC manufacturing...  

- On the cosmic scale, 99 % of observed matter is ionized and therefore in the plasma state.


# Laboratory of Plasma Physics (LPP)

- Natural Plasmas are mostly in space and need in-situ measurements such as:
 - particles (**particles detectors**, spectrometers) 
 - E-field (Electric antenna)
 - B-field (**Search-coils**, Flux-gates)

# Solar Orbiter

- Sun observatory and Solar wind in-situ measurements
- Launch date: end of 2020
- Orbit 0.3 -> 0.8 AU
- Mass total/payload: 1800kg/190kg
- **Power 1100W**
- **Data downlink ~150 kbps**

<div>
<center>
<img src="resources/Solar_Orbiter_payload_annotated_625.jpg" width="800" align="center">
</center>
</div>


# Solar Orbiter

Small amount of telemetry implies heavy onboard computation.

# Radio Plasma Waves instrument (RPW)

- Focuses on waves( **E field** & **B field** ) measurements from **~0Hz** to **16MHz**
- Composed of:
 - **BIAS** 
 - **TNR-HFR**
 - **TDS**
 - **DPU**
 - **LVPS-PDU**
 - **SCM**
 - **LFR** [x]


TODO add MEB spec

# Radio Plasma Waves instrument (RPW)

<div>
<center>
<img src="resources/principe_RPW.png" width="800" align="center">
</center>
</div>


# Low Frequency Receiver (LFR)

- **E** and **B** waves from ~0Hz to 10kHz
- 2W, few kbits 

# LFR processing 